<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/prueba_1_qwen3_8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar dependencias + Imports
!pip install -q -U "transformers>=4.51.0" accelerate safetensors

import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 50.9 MB/s eta 0:00:00


In [2]:
# Token HF
HF_TOKEN = os.environ.get("HF_TOKEN", None)

In [3]:
# Cargar modelo e tokenizer (Qwen3-8B)
# LLM-as-a-Judge binario para concordancia de número en galego

MODEL_ID = "Qwen/Qwen3-8B"

print("🔄 Cargando tokenizer...")
if HF_TOKEN:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, token=HF_TOKEN)
else:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

print("🔄 Cargando modelo (pode tardar uns minutos)...")
if HF_TOKEN:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        token=HF_TOKEN,
        device_map="auto",
        torch_dtype="auto"
    )
else:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        torch_dtype="auto"
    )

model.eval()
print("✅ Modelo cargado correctamente")

🔄 Cargando tokenizer...


config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

OSError: Can't load tokenizer for 'Qwen/Qwen3-8B'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Qwen/Qwen3-8B' is the correct path to a directory containing all relevant files for a Qwen2Tokenizer tokenizer.

In [ ]:
# Función LLM-as-a-Judge (concordancia de número)
def qwen_judge_concordancia(oracion: str) -> str:
    prompt = f"""
Es un LLM-as-a-judge que vai avaliar un modelo de corrección gramatical en galego.

A túa tarefa é avaliar a saída do modelo de corrección gramatical (GEC)
e decidir se a corrección é ADECUADA con respecto á concordancia de número
(singular/plural entre determinante, substantivo, adxectivo ou verbo).

Devolve a resposta EXACTAMENTE co seguinte formato, sen texto adicional:

output_modelo: "<oración avaliada>"
etiqueta: <0 ou 1>
explicacion: "<explicación breve e precisa en galego>"

Criterios:
- etiqueta = 1 → a corrección NON é adecuada (segue habendo erro de concordancia de número)
- etiqueta = 0 → a corrección é adecuada (non hai erro de concordancia de número)
- A explicación debe xustificar só a concordancia de número

NON DEBES, BAIXO NINGÚN CONCEPTO, CORRIXIR O OUTPUT DO MODELO QUE TES QUE AVALIAR. TES QUE LIMITARTE EXCLUSIVAMENTE A DECIDIR SE CONTÉN ERRO DE CONCORDANCIA OU NON.
NON DEBES AVALIAR OUTROS TIPOS DE ERROS.

Agora avalía a seguinte saída dun modelo de GEC:

"<OUTPUT_MODELO>"


"{oracion}"
"""

    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    resposta = tokenizer.decode(
        output[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    )

    return resposta.strip()

In [ ]:
# Probas cos meus exemplos
exemplos = [
    "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas.",
    "O grupos de estudantes que participou no proxecto presentou os resultados finais onte pola tarde.",
    "As propostas que chegaron desde os concellos máis pequenos foi analizadas polo equipo técnico.",
    "A maioría das persoas entrevistadas manifestaron a súa opinión durante a sesión pública."
]

for i, frase in enumerate(exemplos, 1):
    print(f"\n===== EXEMPLO {i} =====")
    print(qwen_judge_concordancia(frase))
